In [17]:
from datetime import datetime,timedelta,date
import time
from pandas.tseries.offsets import BDay
import numpy as np
import pandas as pd
import sqlite3

In [18]:
cols_dict = {"sdate": "日期", "sno": "股票代號","sname":"股票名稱","price":"收盤價"}

In [19]:
con = sqlite3.connect("stock.db")

In [21]:
#同時出現的
sql1 = """
select s.* from selectsys s, selectperson p 
where s.sdate = p.sdate 
and s.sno = p.sno
and s.type == 1
--and s.sdate = '2018-08-09'
"""

dfsys = pd.read_sql(sql1,con)
dfsys.rename(columns=cols_dict,inplace=True)
print(len(dfsys))
dfsys.head()
#dfsys = dfsys[(dfsys['上市上櫃'] == 1) & (dfsys['日期'] >= "2018-06-01")]
#dfsys = dfsys[dfsys['上市上櫃'] == 1]


113


,日期,股票代號,股票名稱,收盤價,type
0,2018-06-08,2881,富邦金,53.80,1
1,2018-06-08,6202,盛群,78.00,1
2,2018-06-11,6202,盛群,79.30,1
3,2018-06-11,6251,定穎,10.20,1
4,2018-06-13,1440,南紡,14.95,1


In [22]:
dfprice = pd.read_sql('select * from price',con)
dfprice = dfprice.loc[:,['交易日','證券代號','證券名稱','收盤價']]
print('read stock price ok')

read stock price ok


In [23]:
def checkScore(df,selecttype='系統-決策樹',checktype='days',caldays=5,endday= ''):    
   
    checkdf = df.copy()
    #checkdf = checkdf[(checkdf['date'] >= '2018-04-21') & (checkdf['股票代號'] == 2501)]
    #checkdf = checkdf[(checkdf['日期'] >= '2018-04-21')]
    selecttype = selecttype
    checktype = checktype # days,sell,today
    print('checktype:',checktype)
    caldays = caldays
    
    print('caldays:',caldays)
    
    global dfempty
    global dfprice
    dfprice1 = pd.DataFrame()
    
    if endday == '':
        endday = datetime.strftime(datetime.now(),'%Y-%m-%d')
        
    today = datetime.strptime(endday,'%Y-%m-%d')

    dateformat = '%Y-%m-%d'
    colNames = ['選股類型','選股日','股票代號','股票名稱','收盤價','買入日','買入收盤價'
                ,'賣出日','賣出收盤價','漲幅','賺','計算類型','計算天數']

    checklist = []

    for index, row in checkdf.iterrows():
        selltype = checktype
        checkdit = {}

        stockno = row['股票代號']
        stockno = str(stockno)
        #print('stockno',stockno)

        stockname = row['股票名稱']
        #print('stockname',stockname)

        price = row['收盤價']
        if type(price) is str:
            price = float(price.replace(",",''))

        #取得系統選股的日期
        selectdate = row['日期']

        #取得系統選股的日期,轉成日期格式
        selectdate = datetime.strptime(selectdate,dateformat)
        selectdatef = selectdate.strftime(dateformat)
        #print('selectdatef',selectdatef)

        #取得買入日
        #buydate = selectdate + BDay(1)
        buydate = np.busday_offset(selectdate, 1, roll='forward',weekmask=weekmask,holidays=holidays)
        buydate = buydate.astype(datetime)
        buydatef = buydate.strftime(dateformat)
        #print('buydatef',buydatef)
        selldatef = ""
        
        #取得賣出日
        #selldate5 = buydate + BDay(5)
        if checktype == 'today':
            selldate = today
        elif checktype == 'sell':
            #從放空的檔案找
            emptydf = dfempty[(dfempty['日期'] > buydatef) & (dfempty['股票代號'] == stockno)]
            if len(emptydf) > 0 :
                emptydate = emptydf.iloc[0,0]
                #print('emptydate:',emptydate)

                selldate = np.busday_offset(emptydate, 1, roll='forward',weekmask=weekmask,holidays=holidays)
                selldate = selldate.astype(datetime)
            else:
                selldate = today
                selltype = 'sell'
        elif checktype == 'max':
            dfprice1 = dfprice.loc[(dfprice['交易日'] > buydatef) & (dfprice['證券代號'] == stockno),:]
            x=dfprice1.max()
            
            selldatef = x[0]
        else:
            selldate = np.busday_offset(buydatef, caldays, roll='forward',weekmask=weekmask,holidays=holidays)
            selldate = selldate.astype(datetime)
        
        if checktype != 'max':
            selldatef = selldate.strftime(dateformat)
            #print('selldatef',selldatef)

        #找出買入日的價格
        buydf = dfprice[(dfprice['交易日'] == buydatef) & (dfprice['證券代號'] == stockno)]
        if len(buydf) > 0 :
            buyprice = buydf.iloc[0,3]
            if (type(buyprice) is str) :
                if buyprice != '--' :
                    buyprice = float(buyprice.replace(",",''))
                else:
                    buyprice = 'not found'
        else:
            buyprice = 'not found'

         #找出賣出日的價格
        selldf = dfprice[(dfprice['交易日'] == selldatef) & (dfprice['證券代號'] == stockno)]
        if len(selldf) > 0 :
            sellprice = selldf.iloc[0,3]
            if type(sellprice) is str:
                if sellprice != '--' :
                    sellprice = float(sellprice.replace(",",''))                    
                else:
                    sellprice = 'not found'
        else:
            sellprice = 'not found'
        #print('check:',stockno,stockname,buydatef,buyprice,selldatef,sellprice)
        
       
        if  buyprice == 'not found' or sellprice == 'not found' :
            diff = 'not found'
            updown = 'not found'
            updatecaldays = 0
        else :
            #計算N日漲幅
            diff = round(((sellprice - buyprice)/buyprice)*100,2)

            if diff >= 1:
                updown = 'Y'
            else:
                updown = 'N'
            #print(buyprice,sellprice,diff,updown)
            caldays = np.busday_count(begindates=buydatef, enddates=selldatef,weekmask=weekmask,holidays=holidays)
            updatecaldays = caldays
            
        if checktype != 'days':
            #caltype = '%s-從買股日至 %s 日' % (selltype,selldatef)
            caltype = '%s' % (selltype)
        else:
            caltype =str(caldays) + '日'


        checkdit[colNames[0]] = selecttype
        checkdit[colNames[1]] = selectdatef
        checkdit[colNames[2]] = stockno
        checkdit[colNames[3]] = stockname
        checkdit[colNames[4]] = price
        checkdit[colNames[5]] = buydatef
        checkdit[colNames[6]] = buyprice
        checkdit[colNames[7]] = selldatef
        checkdit[colNames[8]] = sellprice
        checkdit[colNames[9]] = diff
        checkdit[colNames[10]] = updown
        checkdit[colNames[11]] = caltype
        checkdit[colNames[12]] = updatecaldays


        '''
        checkdit['選股類型'] = '系統-決策樹'
        checkdit['選股日'] = selectdatef
        checkdit['股票代號'] = stockno
        checkdit['股票名稱'] = stockname
        checkdit['收盤價'] = price
        checkdit['買入日'] = buydatef
        checkdit['買入收盤價'] = buyprice
        checkdit['賣出日'] = selldatef
        checkdit['賣出收盤價'] = sellprice
        checkdit['漲幅'] = diff
        checkdit['賺'] = updown
        checkdit['計算類型'] = '5日'
        checkdit['計算天數'] = 5
        '''
        checklist.append(checkdit)
    
    
    checkdf = pd.DataFrame(checklist,columns=colNames)
    return checkdf


In [24]:
# main -----------------------------------
weekmask = [1,1,1,1,1,0,0]
holidays = ['2018-05-01','2018-06-18']
getdate = '2018-08-10'



list_df = []

caldayslist = [1,2,3,4,5,10,15,20]


#by type, check days,today,sell
selecttype2 = 'All'

for d1 in caldayslist:
    getdf1 = checkScore(dfsys,selecttype=selecttype2,checktype='days',caldays=d1)
    list_df.append(getdf1)
    #getdf.head()
    
    
# today
getdf2 = checkScore(dfsys,selecttype=selecttype2,checktype='today',endday=getdate)
list_df.append(getdf2)
#getdf2.head()
    

'''    
# sell
getdf3 = checkScore(dfsys,selecttype=selecttype2,checktype='sell',endday=getdate)
#getdf2.head()
list_df.append(getdf3)
'''

# save to file
dfw1 = pd.concat(list_df)
writer1= pd.ExcelWriter("c:/pythondata/checkSocre_20180607_0809_確認漲幅_同時有.xlsx")
dfw1.to_excel(writer1,'Sheet1',index = False)
writer1.save() 
print('save ok')

checktype: days
caldays: 1
checktype: days
caldays: 2
checktype: days
caldays: 3
checktype: days
caldays: 4
checktype: days
caldays: 5
checktype: days
caldays: 10
checktype: days
caldays: 15
checktype: days
caldays: 20
checktype: today
caldays: 5
save ok


In [25]:
print('== ok ==')
con.commit()
con.close()
print('== close ==')

== ok ==
== close ==


In [26]:
dfan0 = pd.read_excel("c:/pythondata/checkSocre_20180607_0809_確認漲幅_同時有.xlsx")
dfan = dfan0.copy()
print('reak ok')

reak ok


In [27]:
dfan = dfan[dfan['計算天數'] > 0]
dfan['win'] = dfan['賺']
dfan['type'] = dfan['計算類型']
dfan.head()
dfan.loc[dfan['win'] != 'Y','win'] = 0
dfan.loc[dfan['win'] == 'Y','win'] = 1
dfan['漲幅'] = dfan['漲幅'].astype(float)
dfan.pivot_table('win', index='選股類型',columns='type')

type,10日,15日,1日,20日,2日,3日,4日,5日,today
選股類型,,,,,,,,,
All,0.384615,0.483333,0.201835,0.416667,0.259615,0.323232,0.364583,0.373626,0.33945
